# LangGraph Multi-Agent System for Payment Processing

This notebook demonstrates a **multi-agent payment processing system** using LangGraph with:

- **Supervisor Agent** - Orchestrates workflow and routes to specialized agents
- **Compliance Agent** - Checks sanctions, AML, and regulatory requirements
- **Fraud Detection Agent** - Analyzes transaction patterns for fraud indicators
- **Risk Assessment Agent** - Evaluates overall payment risk
- **Shared Tools** - Policy retrieval, customer lookup, sanctions check

The focus is on:
- Multi-agent orchestration with LangGraph
- Agent collaboration and information sharing
- Structured state management
- Safety guardrails (AI assists, never decides)

## Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────────────────────────┐
│                         MULTI-AGENT PAYMENT PROCESSING SYSTEM                           │
└─────────────────────────────────────────────────────────────────────────────────────────┘

                                    ┌───────────────────┐
                                    │  Payment Request  │
                                    │     (Input)       │
                                    └─────────┬─────────┘
                                              │
                                              ▼
    ┌────────────────────────┐  ┌────────────────────────┐  ┌────────────────────────┐
    │   COMPLIANCE AGENT     │  │  FRAUD DETECTION AGENT │  │  RISK ASSESSMENT AGENT │
    │  • Sanctions check     │  │  • Pattern analysis    │  │  • Risk scoring        │
    │  • AML verification    │  │  • Velocity checks     │  │  • Combines findings   │
    │  • Regulatory rules    │  │  • Anomaly detection   │  │  • from other agents   │
    │          │             │  │          │             │  │                        │
    │          ▼             │  │          ▼             │  │  (no tools, uses       │
    │  ┌─────────────────┐   │  │  ┌─────────────────┐   │  │   compliance_result    │
    │  │ SanctionsDB     │   │  │  │ CustomerDB      │   │  │   and fraud_result)    │
    │  │ PolicyDB        │   │  │  └─────────────────┘   │  │                        │
    │  └─────────────────┘   │  │        TOOL            │  │                        │
    │       TOOLS            │  │                        │  │                        │
    └────────────┬───────────┘  └────────────┬───────────┘  └────────────┬───────────┘
                 │                           │                           │
                 └───────────────────────────┼───────────────────────────┘
                                             │
                                             ▼
                              ┌───────────────────────────────┐
                              │      SUPERVISOR AGENT         │
                              │  • Aggregates all findings    │
                              │  • Identifies conflicts       │
                              │  • Makes final recommendation │
                              │  (no tools, synthesizes only) │
                              └───────────────────────────────┘
                                             │
                                             ▼
                              ┌───────────────────────────────┐
                              │   ANALYST DASHBOARD           │
                              │   (Human Decision)            │
                              └───────────────────────────────┘
```

**Key principle:** Agents collaborate and share information, but ONLY the human analyst makes final payment decisions.

## 1. Install Dependencies

**Prerequisites**: Ollama installed with a model.
```bash
ollama pull llama3.1:8b
```

In [1]:
!pip install langchain langchain-ollama langgraph pydantic


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## 2. Import Libraries

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated, List, Optional, Literal
from enum import Enum
import operator
import json
from datetime import datetime

print("Libraries imported successfully!")

Libraries imported successfully!


## 3. Define State Schema

The state is shared across all agents and accumulates information as the workflow progresses.

In [3]:
class PaymentType(str, Enum):
    WIRE_TRANSFER = "wire_transfer"
    CARD_PAYMENT = "card_payment"
    ACH_TRANSFER = "ach_transfer"
    INSTANT_PAYMENT = "instant_payment"

class AgentFinding(BaseModel):
    agent_name: str
    finding_type: str
    severity: Literal["low", "medium", "high", "critical"]
    description: str
    confidence: float = Field(ge=0.0, le=1.0)

class PaymentRequest(BaseModel):
    transaction_id: str
    amount: float
    currency: str
    payment_type: PaymentType
    sender_id: str
    sender_name: str
    sender_country: str
    receiver_id: str
    receiver_name: str
    receiver_country: str
    purpose: str
    sender_account_age_days: int = 0
    is_first_transaction: bool = False

class MultiAgentState(TypedDict):
    payment_request: dict
    messages: Annotated[List[str], operator.add]
    compliance_result: Optional[dict]
    fraud_result: Optional[dict]
    risk_result: Optional[dict]
    agent_findings: Annotated[List[dict], operator.add]
    final_recommendation: Optional[dict]
    current_agent: str
    workflow_complete: bool

print("State schema defined!")

State schema defined!


## 4. Define Shared Tools

Tools that can be used by multiple agents.

In [4]:
class SanctionsDatabase:
    SANCTIONED_COUNTRIES = ["KP", "IR", "SY", "CU"]
    SANCTIONED_ENTITIES = ["BadActor Corp", "Shady LLC", "Suspicious Inc"]
    
    @classmethod
    def check_sanctions(cls, country: str, entity_name: str) -> dict:
        country_sanctioned = country.upper() in cls.SANCTIONED_COUNTRIES
        entity_sanctioned = any(s.lower() in entity_name.lower() for s in cls.SANCTIONED_ENTITIES)
        return {
            "country_sanctioned": country_sanctioned,
            "entity_sanctioned": entity_sanctioned,
            "sanctions_hit": country_sanctioned or entity_sanctioned,
            "details": f"Country: {country} ({'SANCTIONED' if country_sanctioned else 'OK'}), Entity: {entity_name} ({'SANCTIONED' if entity_sanctioned else 'OK'})"
        }

class PolicyDatabase:
    POLICIES = {
        "high_value": {
            "threshold": 10000,
            "rule": "Transactions above $10,000 require enhanced due diligence (Policy AML-4.2)"
        },
        "cross_border": {
            "high_risk_countries": ["NG", "RU", "CN", "VN"],
            "rule": "Cross-border to high-risk countries require additional verification (Policy CB-3.1)"
        },
        "new_account": {
            "threshold_days": 30,
            "rule": "Accounts less than 30 days old require manual review (Policy ACC-2.3)"
        },
        "first_transaction": {
            "rule": "First-time transactions require additional verification (Policy FT-1.1)"
        }
    }
    
    @classmethod
    def get_applicable_policies(cls, payment: dict) -> List[dict]:
        applicable = []
        if payment.get("amount", 0) > cls.POLICIES["high_value"]["threshold"]:
            applicable.append({"policy": "high_value", "rule": cls.POLICIES["high_value"]["rule"]})
        if payment.get("receiver_country") in cls.POLICIES["cross_border"]["high_risk_countries"]:
            applicable.append({"policy": "cross_border", "rule": cls.POLICIES["cross_border"]["rule"]})
        if payment.get("sender_account_age_days", 0) < cls.POLICIES["new_account"]["threshold_days"]:
            applicable.append({"policy": "new_account", "rule": cls.POLICIES["new_account"]["rule"]})
        if payment.get("is_first_transaction", False):
            applicable.append({"policy": "first_transaction", "rule": cls.POLICIES["first_transaction"]["rule"]})
        return applicable

class CustomerDatabase:
    CUSTOMERS = {
        "CUST001": {"risk_score": 0.2, "transaction_count": 150, "avg_transaction": 500, "account_status": "good"},
        "CUST002": {"risk_score": 0.7, "transaction_count": 5, "avg_transaction": 8000, "account_status": "review"},
        "CUST003": {"risk_score": 0.1, "transaction_count": 500, "avg_transaction": 200, "account_status": "good"},
    }
    
    @classmethod
    def get_customer_profile(cls, customer_id: str) -> dict:
        return cls.CUSTOMERS.get(customer_id, {
            "risk_score": 0.5,
            "transaction_count": 0,
            "avg_transaction": 0,
            "account_status": "new"
        })

print("Shared tools defined:")
print("  - SanctionsDatabase")
print("  - PolicyDatabase")
print("  - CustomerDatabase")

Shared tools defined:
  - SanctionsDatabase
  - PolicyDatabase
  - CustomerDatabase


In [5]:
print("Testing tools...")
print("\nSanctions check (US, John Doe):", SanctionsDatabase.check_sanctions("US", "John Doe"))
print("Sanctions check (IR, Test Corp):", SanctionsDatabase.check_sanctions("IR", "Test Corp"))
print("\nPolicies for $15000 to NG:", PolicyDatabase.get_applicable_policies({"amount": 15000, "receiver_country": "NG"}))
print("\nCustomer profile CUST001:", CustomerDatabase.get_customer_profile("CUST001"))

Testing tools...

Sanctions check (US, John Doe): {'country_sanctioned': False, 'entity_sanctioned': False, 'sanctions_hit': False, 'details': 'Country: US (OK), Entity: John Doe (OK)'}
Sanctions check (IR, Test Corp): {'country_sanctioned': True, 'entity_sanctioned': False, 'sanctions_hit': True, 'details': 'Country: IR (SANCTIONED), Entity: Test Corp (OK)'}

Policies for $15000 to NG: [{'policy': 'high_value', 'rule': 'Transactions above $10,000 require enhanced due diligence (Policy AML-4.2)'}, {'policy': 'cross_border', 'rule': 'Cross-border to high-risk countries require additional verification (Policy CB-3.1)'}, {'policy': 'new_account', 'rule': 'Accounts less than 30 days old require manual review (Policy ACC-2.3)'}]

Customer profile CUST001: {'risk_score': 0.2, 'transaction_count': 150, 'avg_transaction': 500, 'account_status': 'good'}


## 5. Setup LLM

In [7]:
llm = ChatOllama(model='llama3.1:8b', temperature=0)
print(f"LLM configured: {llm.model}")

LLM configured: llama3.1:8b


## 6. Define Agent Prompts

In [8]:
COMPLIANCE_AGENT_PROMPT = """
You are the Compliance Agent in a multi-agent payment processing system.

Your role is to check regulatory compliance for the payment request.
You have access to sanctions check results and applicable policies.

IMPORTANT: You are an ASSISTANT. You do NOT approve, reject, or make payment decisions.
You only analyze and report findings.

Payment Request:
{payment_request}

Sanctions Check Result:
{sanctions_result}

Applicable Policies:
{applicable_policies}

Analyze the compliance status and provide your findings as JSON:
{{
    "compliance_status": "compliant" | "review_required" | "non_compliant",
    "sanctions_clear": true | false,
    "policy_violations": [list of policy violations],
    "findings": [list of specific findings],
    "risk_factors": [list of compliance risk factors],
    "summary": "brief summary of compliance analysis"
}}

Respond with valid JSON only.
"""

FRAUD_AGENT_PROMPT = """
You are the Fraud Detection Agent in a multi-agent payment processing system.

Your role is to analyze the payment for potential fraud indicators.
You have access to the sender's profile and transaction patterns.

IMPORTANT: You are an ASSISTANT. You do NOT approve, reject, or make payment decisions.
You only analyze and report findings.

Payment Request:
{payment_request}

Sender Profile:
{sender_profile}

Analyze for fraud indicators and provide your findings as JSON:
{{
    "fraud_risk_level": "low" | "medium" | "high" | "critical",
    "fraud_score": 0.0-1.0,
    "fraud_indicators": [
        {{"indicator": "description", "severity": "low|medium|high", "confidence": 0.0-1.0}}
    ],
    "velocity_check": {{"passed": true|false, "reason": "explanation"}},
    "pattern_analysis": "analysis of transaction patterns",
    "summary": "brief summary of fraud analysis"
}}

Respond with valid JSON only.
"""

RISK_AGENT_PROMPT = """
You are the Risk Assessment Agent in a multi-agent payment processing system.

Your role is to provide an overall risk assessment combining all findings.
You have access to compliance and fraud detection results.

IMPORTANT: You are an ASSISTANT. You do NOT approve, reject, or make payment decisions.
You only assess risk and recommend next steps for human review.

Payment Request:
{payment_request}

Compliance Agent Findings:
{compliance_result}

Fraud Agent Findings:
{fraud_result}

Provide your risk assessment as JSON:
{{
    "overall_risk_level": "low" | "medium" | "high" | "critical",
    "overall_risk_score": 0.0-1.0,
    "risk_factors": [
        {{"factor": "description", "weight": 0.0-1.0, "source": "compliance|fraud|combined"}}
    ],
    "recommended_action": "proceed_with_monitoring" | "request_additional_verification" | "escalate_to_senior_analyst" | "flag_for_investigation",
    "confidence": 0.0-1.0,
    "summary": "comprehensive risk summary for analyst"
}}

Respond with valid JSON only.
"""

SUPERVISOR_PROMPT = """
You are the Supervisor Agent coordinating a multi-agent payment processing system.

Your role is to:
1. Summarize findings from all specialist agents
2. Identify any conflicts or areas needing attention
3. Provide a final recommendation for the human analyst

IMPORTANT: You are an ASSISTANT. You do NOT approve, reject, or make payment decisions.
The human analyst makes all final decisions.

Payment Request:
{payment_request}

Compliance Agent Result:
{compliance_result}

Fraud Agent Result:
{fraud_result}

Risk Assessment Result:
{risk_result}

Provide your final summary as JSON:
{{
    "transaction_id": "the transaction ID",
    "executive_summary": "2-3 sentence summary for analyst",
    "overall_risk_level": "low" | "medium" | "high" | "critical",
    "key_findings": ["list of most important findings"],
    "areas_of_concern": ["specific concerns requiring attention"],
    "recommended_action": "proceed_with_monitoring" | "request_additional_verification" | "escalate_to_senior_analyst" | "flag_for_investigation",
    "agent_agreement": {{"all_agree": true|false, "disagreements": []}},
    "additional_info_needed": ["any additional information that would help"]
}}

Respond with valid JSON only.
"""

print("Agent prompts defined!")

Agent prompts defined!


## 7. Define Agent Functions

In [9]:
import re

def extract_json_from_response(response: str) -> dict:
    json_match = re.search(r'```(?:json)?\s*([\s\S]*?)```', response)
    if json_match:
        json_str = json_match.group(1).strip()
    else:
        json_str = response.strip()
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON", "raw_response": response}

def compliance_agent(state: MultiAgentState) -> MultiAgentState:
    payment = state["payment_request"]
    
    sanctions_result = SanctionsDatabase.check_sanctions(
        payment.get("receiver_country", ""),
        payment.get("receiver_name", "")
    )
    sender_sanctions = SanctionsDatabase.check_sanctions(
        payment.get("sender_country", ""),
        payment.get("sender_name", "")
    )
    
    applicable_policies = PolicyDatabase.get_applicable_policies(payment)
    
    prompt = COMPLIANCE_AGENT_PROMPT.format(
        payment_request=json.dumps(payment, indent=2),
        sanctions_result=json.dumps({"sender": sender_sanctions, "receiver": sanctions_result}, indent=2),
        applicable_policies=json.dumps(applicable_policies, indent=2)
    )
    
    response = llm.invoke(prompt)
    result = extract_json_from_response(response.content)
    
    findings = []
    if sanctions_result.get("sanctions_hit") or sender_sanctions.get("sanctions_hit"):
        findings.append({
            "agent_name": "compliance_agent",
            "finding_type": "sanctions_hit",
            "severity": "critical",
            "description": "Sanctions screening hit detected",
            "confidence": 1.0
        })
    
    for policy in applicable_policies:
        findings.append({
            "agent_name": "compliance_agent",
            "finding_type": "policy_applicable",
            "severity": "medium",
            "description": policy["rule"],
            "confidence": 1.0
        })
    
    return {
        "compliance_result": result,
        "agent_findings": findings,
        "messages": [f"Compliance Agent completed analysis"],
        "current_agent": "fraud_agent"
    }

def fraud_agent(state: MultiAgentState) -> MultiAgentState:
    payment = state["payment_request"]
    
    sender_profile = CustomerDatabase.get_customer_profile(payment.get("sender_id", ""))
    
    prompt = FRAUD_AGENT_PROMPT.format(
        payment_request=json.dumps(payment, indent=2),
        sender_profile=json.dumps(sender_profile, indent=2)
    )
    
    response = llm.invoke(prompt)
    result = extract_json_from_response(response.content)
    
    findings = []
    fraud_indicators = result.get("fraud_indicators", [])
    for indicator in fraud_indicators:
        if isinstance(indicator, dict):
            findings.append({
                "agent_name": "fraud_agent",
                "finding_type": "fraud_indicator",
                "severity": indicator.get("severity", "medium"),
                "description": indicator.get("indicator", "Unknown indicator"),
                "confidence": indicator.get("confidence", 0.5)
            })
    
    return {
        "fraud_result": result,
        "agent_findings": findings,
        "messages": [f"Fraud Detection Agent completed analysis"],
        "current_agent": "risk_agent"
    }

def risk_agent(state: MultiAgentState) -> MultiAgentState:
    payment = state["payment_request"]
    compliance_result = state.get("compliance_result", {})
    fraud_result = state.get("fraud_result", {})
    
    prompt = RISK_AGENT_PROMPT.format(
        payment_request=json.dumps(payment, indent=2),
        compliance_result=json.dumps(compliance_result, indent=2),
        fraud_result=json.dumps(fraud_result, indent=2)
    )
    
    response = llm.invoke(prompt)
    result = extract_json_from_response(response.content)
    
    findings = [{
        "agent_name": "risk_agent",
        "finding_type": "risk_assessment",
        "severity": result.get("overall_risk_level", "medium"),
        "description": result.get("summary", "Risk assessment completed"),
        "confidence": result.get("confidence", 0.5)
    }]
    
    return {
        "risk_result": result,
        "agent_findings": findings,
        "messages": [f"Risk Assessment Agent completed analysis"],
        "current_agent": "supervisor"
    }

def supervisor_agent(state: MultiAgentState) -> MultiAgentState:
    payment = state["payment_request"]
    compliance_result = state.get("compliance_result", {})
    fraud_result = state.get("fraud_result", {})
    risk_result = state.get("risk_result", {})
    
    prompt = SUPERVISOR_PROMPT.format(
        payment_request=json.dumps(payment, indent=2),
        compliance_result=json.dumps(compliance_result, indent=2),
        fraud_result=json.dumps(fraud_result, indent=2),
        risk_result=json.dumps(risk_result, indent=2)
    )
    
    response = llm.invoke(prompt)
    result = extract_json_from_response(response.content)
    
    return {
        "final_recommendation": result,
        "messages": [f"Supervisor Agent completed final review"],
        "current_agent": "complete",
        "workflow_complete": True
    }

print("Agent functions defined!")

Agent functions defined!


## 8. Build the Multi-Agent Graph

In [10]:
def route_to_next(state: MultiAgentState) -> str:
    current = state.get("current_agent", "compliance_agent")
    if current == "fraud_agent":
        return "fraud_agent"
    elif current == "risk_agent":
        return "risk_agent"
    elif current == "supervisor":
        return "supervisor"
    elif current == "complete":
        return END
    return "compliance_agent"

workflow = StateGraph(MultiAgentState)

workflow.add_node("compliance_agent", compliance_agent)
workflow.add_node("fraud_agent", fraud_agent)
workflow.add_node("risk_agent", risk_agent)
workflow.add_node("supervisor", supervisor_agent)

workflow.set_entry_point("compliance_agent")

workflow.add_conditional_edges(
    "compliance_agent",
    route_to_next,
    {
        "fraud_agent": "fraud_agent",
        "risk_agent": "risk_agent",
        "supervisor": "supervisor",
        END: END
    }
)

workflow.add_conditional_edges(
    "fraud_agent",
    route_to_next,
    {
        "risk_agent": "risk_agent",
        "supervisor": "supervisor",
        END: END
    }
)

workflow.add_conditional_edges(
    "risk_agent",
    route_to_next,
    {
        "supervisor": "supervisor",
        END: END
    }
)

workflow.add_edge("supervisor", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("Multi-agent graph compiled!")
print("\nWorkflow: compliance_agent -> fraud_agent -> risk_agent -> supervisor -> END")

Multi-agent graph compiled!

Workflow: compliance_agent -> fraud_agent -> risk_agent -> supervisor -> END


## 9. Create Test Payment Requests

In [11]:
high_risk_payment = {
    "transaction_id": "PAY-2024-001",
    "amount": 25000.00,
    "currency": "USD",
    "payment_type": "wire_transfer",
    "sender_id": "CUST002",
    "sender_name": "John Smith",
    "sender_country": "US",
    "receiver_id": "RCV001",
    "receiver_name": "International Trading Ltd",
    "receiver_country": "NG",
    "purpose": "Business payment for goods",
    "sender_account_age_days": 15,
    "is_first_transaction": True
}

low_risk_payment = {
    "transaction_id": "PAY-2024-002",
    "amount": 250.00,
    "currency": "USD",
    "payment_type": "card_payment",
    "sender_id": "CUST001",
    "sender_name": "Jane Doe",
    "sender_country": "US",
    "receiver_id": "RCV002",
    "receiver_name": "Local Coffee Shop",
    "receiver_country": "US",
    "purpose": "Coffee purchase",
    "sender_account_age_days": 365,
    "is_first_transaction": False
}

sanctions_payment = {
    "transaction_id": "PAY-2024-003",
    "amount": 5000.00,
    "currency": "EUR",
    "payment_type": "wire_transfer",
    "sender_id": "CUST003",
    "sender_name": "Bob Wilson",
    "sender_country": "DE",
    "receiver_id": "RCV003",
    "receiver_name": "Tehran Imports",
    "receiver_country": "IR",
    "purpose": "Import payment",
    "sender_account_age_days": 200,
    "is_first_transaction": False
}

print("Test payment requests created:")
print(f"  1. High Risk: {high_risk_payment['transaction_id']} - ${high_risk_payment['amount']} to {high_risk_payment['receiver_country']}")
print(f"  2. Low Risk: {low_risk_payment['transaction_id']} - ${low_risk_payment['amount']} to {low_risk_payment['receiver_country']}")
print(f"  3. Sanctions Hit: {sanctions_payment['transaction_id']} - €{sanctions_payment['amount']} to {sanctions_payment['receiver_country']}")

Test payment requests created:
  1. High Risk: PAY-2024-001 - $25000.0 to NG
  2. Low Risk: PAY-2024-002 - $250.0 to US
  3. Sanctions Hit: PAY-2024-003 - €5000.0 to IR


## 10. Execute Multi-Agent Workflow - High Risk Payment

In [12]:
print("=" * 70)
print("MULTI-AGENT WORKFLOW: High Risk Payment")
print("=" * 70)
print(f"\nTransaction: {high_risk_payment['transaction_id']}")
print(f"Amount: ${high_risk_payment['amount']:,.2f} {high_risk_payment['currency']}")
print(f"Route: {high_risk_payment['sender_country']} -> {high_risk_payment['receiver_country']}")
print("\n" + "-" * 70)

initial_state = {
    "payment_request": high_risk_payment,
    "messages": [],
    "compliance_result": None,
    "fraud_result": None,
    "risk_result": None,
    "agent_findings": [],
    "final_recommendation": None,
    "current_agent": "compliance_agent",
    "workflow_complete": False
}

config = {"configurable": {"thread_id": "high_risk_001"}}

print("\nProcessing through agents...")
for event in app.stream(initial_state, config):
    for node_name, node_output in event.items():
        if "messages" in node_output and node_output["messages"]:
            print(f"  ✓ {node_output['messages'][-1]}")

MULTI-AGENT WORKFLOW: High Risk Payment

Transaction: PAY-2024-001
Amount: $25,000.00 USD
Route: US -> NG

----------------------------------------------------------------------

Processing through agents...
  ✓ Compliance Agent completed analysis
  ✓ Fraud Detection Agent completed analysis
  ✓ Risk Assessment Agent completed analysis
  ✓ Supervisor Agent completed final review


In [13]:
final_state = app.get_state(config)

print("\n" + "=" * 70)
print("AGENT RESULTS")
print("=" * 70)

print("\n📋 COMPLIANCE AGENT:")
print(json.dumps(final_state.values.get("compliance_result", {}), indent=2))

print("\n🔍 FRAUD DETECTION AGENT:")
print(json.dumps(final_state.values.get("fraud_result", {}), indent=2))

print("\n⚠️ RISK ASSESSMENT AGENT:")
print(json.dumps(final_state.values.get("risk_result", {}), indent=2))


AGENT RESULTS

📋 COMPLIANCE AGENT:
{
  "compliance_status": "review_required",
  "sanctions_clear": true,
  "policy_violations": [
    {
      "policy": "high_value",
      "rule": "Transactions above $10,000 require enhanced due diligence (Policy AML-4.2)"
    },
    {
      "policy": "cross_border",
      "rule": "Cross-border to high-risk countries require additional verification (Policy CB-3.1)"
    }
  ],
  "findings": [
    "Transaction amount exceeds $10,000 threshold.",
    "Payment is cross-border to a country with potential risk (NG)."
  ],
  "risk_factors": [
    "High-value transaction",
    "Cross-border payment to high-risk country"
  ],
  "summary": "The payment request requires review due to high-value and cross-border factors."
}

🔍 FRAUD DETECTION AGENT:
{
  "fraud_risk_level": "medium",
  "fraud_score": 0.6,
  "fraud_indicators": [
    {
      "indicator": "High transaction amount exceeds average by 312%",
      "severity": "high",
      "confidence": 0.8
    },
   

In [14]:
print("\n" + "=" * 70)
print("SUPERVISOR FINAL RECOMMENDATION")
print("=" * 70)

final_rec = final_state.values.get("final_recommendation", {})
print(json.dumps(final_rec, indent=2))

print("\n" + "=" * 70)
print("ALL AGENT FINDINGS")
print("=" * 70)

for finding in final_state.values.get("agent_findings", []):
    severity_emoji = {"low": "🟢", "medium": "🟡", "high": "🟠", "critical": "🔴"}.get(finding.get("severity"), "⚪")
    print(f"  {severity_emoji} [{finding.get('agent_name')}] {finding.get('description')}")


SUPERVISOR FINAL RECOMMENDATION
{
  "transaction_id": "PAY-2024-001",
  "executive_summary": "This payment request requires review due to high-value and cross-border factors, unusually large transaction amount, and a sender with a higher-than-average risk score. Further verification is recommended to ensure the legitimacy of this transaction.",
  "overall_risk_level": "high",
  "key_findings": [
    "Transaction amount exceeds $10,000 threshold.",
    "Payment is cross-border to a country with potential risk (NG).",
    "Unusually large transaction amount exceeds sender's average by more than 3 times."
  ],
  "areas_of_concern": [
    "High-value and cross-border factors",
    "Sender has a higher-than-average risk score"
  ],
  "recommended_action": "request_additional_verification",
  "agent_agreement": {
    "all_agree": false,
    "disagreements": [
      "Compliance Agent: Review required due to high-value and cross-border factors.",
      "Fraud Agent: Further review recommended

## 11. Execute Multi-Agent Workflow - Low Risk Payment

In [15]:
print("=" * 70)
print("MULTI-AGENT WORKFLOW: Low Risk Payment")
print("=" * 70)
print(f"\nTransaction: {low_risk_payment['transaction_id']}")
print(f"Amount: ${low_risk_payment['amount']:,.2f} {low_risk_payment['currency']}")
print(f"Route: {low_risk_payment['sender_country']} -> {low_risk_payment['receiver_country']}")
print("\n" + "-" * 70)

initial_state_low = {
    "payment_request": low_risk_payment,
    "messages": [],
    "compliance_result": None,
    "fraud_result": None,
    "risk_result": None,
    "agent_findings": [],
    "final_recommendation": None,
    "current_agent": "compliance_agent",
    "workflow_complete": False
}

config_low = {"configurable": {"thread_id": "low_risk_001"}}

print("\nProcessing through agents...")
for event in app.stream(initial_state_low, config_low):
    for node_name, node_output in event.items():
        if "messages" in node_output and node_output["messages"]:
            print(f"  ✓ {node_output['messages'][-1]}")

final_state_low = app.get_state(config_low)

print("\n" + "=" * 70)
print("SUPERVISOR FINAL RECOMMENDATION")
print("=" * 70)
print(json.dumps(final_state_low.values.get("final_recommendation", {}), indent=2))

MULTI-AGENT WORKFLOW: Low Risk Payment

Transaction: PAY-2024-002
Amount: $250.00 USD
Route: US -> US

----------------------------------------------------------------------

Processing through agents...
  ✓ Compliance Agent completed analysis
  ✓ Fraud Detection Agent completed analysis
  ✓ Risk Assessment Agent completed analysis
  ✓ Supervisor Agent completed final review

SUPERVISOR FINAL RECOMMENDATION
{
  "transaction_id": "PAY-2024-002",
  "executive_summary": "The transaction appears to be compliant and low-risk, but ongoing monitoring is recommended due to minimal indicators of potential issues.",
  "overall_risk_level": "low",
  "key_findings": [
    "Compliance check indicates no sanctions or policy violations.",
    "Fraud risk level is low with a score of 0.1.",
    "Risk assessment recommends proceeding with monitoring."
  ],
  "areas_of_concern": [],
  "recommended_action": "proceed_with_monitoring",
  "agent_agreement": {
    "all_agree": true,
    "disagreements": []
 

## 12. Execute Multi-Agent Workflow - Sanctions Hit

In [16]:
print("=" * 70)
print("MULTI-AGENT WORKFLOW: Sanctions Hit Payment")
print("=" * 70)
print(f"\nTransaction: {sanctions_payment['transaction_id']}")
print(f"Amount: €{sanctions_payment['amount']:,.2f} {sanctions_payment['currency']}")
print(f"Route: {sanctions_payment['sender_country']} -> {sanctions_payment['receiver_country']}")
print(f"⚠️  Receiver Country: {sanctions_payment['receiver_country']} (SANCTIONED)")
print("\n" + "-" * 70)

initial_state_sanc = {
    "payment_request": sanctions_payment,
    "messages": [],
    "compliance_result": None,
    "fraud_result": None,
    "risk_result": None,
    "agent_findings": [],
    "final_recommendation": None,
    "current_agent": "compliance_agent",
    "workflow_complete": False
}

config_sanc = {"configurable": {"thread_id": "sanctions_001"}}

print("\nProcessing through agents...")
for event in app.stream(initial_state_sanc, config_sanc):
    for node_name, node_output in event.items():
        if "messages" in node_output and node_output["messages"]:
            print(f"  ✓ {node_output['messages'][-1]}")

final_state_sanc = app.get_state(config_sanc)

print("\n" + "=" * 70)
print("🚨 CRITICAL FINDINGS")
print("=" * 70)

for finding in final_state_sanc.values.get("agent_findings", []):
    if finding.get("severity") == "critical":
        print(f"  🔴 [{finding.get('agent_name')}] {finding.get('description')}")

print("\n" + "=" * 70)
print("SUPERVISOR FINAL RECOMMENDATION")
print("=" * 70)
print(json.dumps(final_state_sanc.values.get("final_recommendation", {}), indent=2))

MULTI-AGENT WORKFLOW: Sanctions Hit Payment

Transaction: PAY-2024-003
Amount: €5,000.00 EUR
Route: DE -> IR
⚠️  Receiver Country: IR (SANCTIONED)

----------------------------------------------------------------------

Processing through agents...
  ✓ Compliance Agent completed analysis
  ✓ Fraud Detection Agent completed analysis
  ✓ Risk Assessment Agent completed analysis
  ✓ Supervisor Agent completed final review

🚨 CRITICAL FINDINGS
  🔴 [compliance_agent] Sanctions screening hit detected

SUPERVISOR FINAL RECOMMENDATION
{
  "transaction_id": "PAY-2024-003",
  "executive_summary": "Payment request from Bob Wilson to Tehran Imports has been flagged for investigation due to sanctions on receiver's country and high transaction amount. Compliance and risk assessment agents have identified potential issues, while the fraud agent indicates a low risk level.",
  "overall_risk_level": "high",
  "key_findings": [
    "Receiver's country (IR) is sanctioned.",
    "Transaction amount is hig

## 13. Side-by-Side Comparison

In [17]:
print("=" * 70)
print("PAYMENT ANALYSIS COMPARISON")
print("=" * 70)

results = [
    ("High Risk", high_risk_payment, final_state.values),
    ("Low Risk", low_risk_payment, final_state_low.values),
    ("Sanctions", sanctions_payment, final_state_sanc.values),
]

print(f"\n{'Payment':<15} {'Amount':>12} {'Route':<10} {'Risk Level':<12} {'Recommended Action'}")
print("-" * 80)

for name, payment, state in results:
    final_rec = state.get("final_recommendation", {})
    risk_level = final_rec.get("overall_risk_level", "N/A")
    action = final_rec.get("recommended_action", "N/A")
    route = f"{payment['sender_country']}->{payment['receiver_country']}"
    amount = f"${payment['amount']:,.0f}"
    print(f"{name:<15} {amount:>12} {route:<10} {risk_level:<12} {action}")

PAYMENT ANALYSIS COMPARISON

Payment               Amount Route      Risk Level   Recommended Action
--------------------------------------------------------------------------------
High Risk            $25,000 US->NG     high         request_additional_verification
Low Risk                $250 US->US     low          proceed_with_monitoring
Sanctions             $5,000 DE->IR     high         flag_for_investigation


## 14. Complete Audit Log

In [18]:
def generate_audit_log(payment: dict, state_values: dict) -> dict:
    return {
        "timestamp": datetime.now().isoformat(),
        "transaction_id": payment["transaction_id"],
        "workflow": "multi_agent_payment_processing",
        "agents_invoked": ["compliance_agent", "fraud_agent", "risk_agent", "supervisor"],
        "payment_summary": {
            "amount": payment["amount"],
            "currency": payment["currency"],
            "sender_country": payment["sender_country"],
            "receiver_country": payment["receiver_country"]
        },
        "findings_count": len(state_values.get("agent_findings", [])),
        "final_recommendation": state_values.get("final_recommendation", {}),
        "metadata": {
            "llm_model": "llama3.1:8b",
            "framework": "langgraph",
            "architecture": "multi_agent_supervisor"
        }
    }

print("=" * 70)
print("AUDIT LOG - High Risk Payment")
print("=" * 70)
audit = generate_audit_log(high_risk_payment, final_state.values)
print(json.dumps(audit, indent=2, default=str))

AUDIT LOG - High Risk Payment
{
  "timestamp": "2025-12-17T15:47:45.516998",
  "transaction_id": "PAY-2024-001",
  "workflow": "multi_agent_payment_processing",
  "agents_invoked": [
    "compliance_agent",
    "fraud_agent",
    "risk_agent",
    "supervisor"
  ],
  "payment_summary": {
    "amount": 25000.0,
    "currency": "USD",
    "sender_country": "US",
    "receiver_country": "NG"
  },
  "findings_count": 7,
  "final_recommendation": {
    "transaction_id": "PAY-2024-001",
    "executive_summary": "This payment request requires review due to high-value and cross-border factors, unusually large transaction amount, and a sender with a higher-than-average risk score. Further verification is recommended to ensure the legitimacy of this transaction.",
    "overall_risk_level": "high",
    "key_findings": [
      "Transaction amount exceeds $10,000 threshold.",
      "Payment is cross-border to a country with potential risk (NG).",
      "Unusually large transaction amount exceeds se

## 15. Safety Validation

In [19]:
FORBIDDEN_PHRASES = [
    "approve the payment",
    "reject the payment",
    "block the transaction",
    "cancel the payment",
    "authorize the transfer",
    "i have decided",
    "i will approve",
    "i will reject"
]

def validate_agent_outputs(state_values: dict) -> dict:
    violations = []
    outputs_to_check = [
        ("compliance_result", state_values.get("compliance_result", {})),
        ("fraud_result", state_values.get("fraud_result", {})),
        ("risk_result", state_values.get("risk_result", {})),
        ("final_recommendation", state_values.get("final_recommendation", {}))
    ]
    
    for name, output in outputs_to_check:
        output_str = json.dumps(output).lower()
        for phrase in FORBIDDEN_PHRASES:
            if phrase in output_str:
                violations.append({"agent": name, "phrase": phrase})
    
    return {
        "is_safe": len(violations) == 0,
        "violations": violations,
        "message": "All agent outputs are safe" if not violations else f"Found {len(violations)} violations"
    }

print("=" * 70)
print("SAFETY VALIDATION")
print("=" * 70)

for name, payment, state in results:
    validation = validate_agent_outputs(state)
    status = "✅ SAFE" if validation["is_safe"] else "❌ VIOLATIONS FOUND"
    print(f"\n{name} Payment: {status}")
    if not validation["is_safe"]:
        for v in validation["violations"]:
            print(f"  - {v['agent']}: '{v['phrase']}'")

SAFETY VALIDATION

High Risk Payment: ✅ SAFE

Low Risk Payment: ✅ SAFE

Sanctions Payment: ✅ SAFE


## Summary

This notebook demonstrated a **multi-agent payment processing system** using LangGraph:

### Architecture Components

| Agent | Role | Key Checks |
|-------|------|------------|
| **Compliance Agent** | Regulatory compliance | Sanctions, AML, Policies |
| **Fraud Agent** | Fraud detection | Patterns, Velocity, Anomalies |
| **Risk Agent** | Risk assessment | Combined risk scoring |
| **Supervisor** | Orchestration | Aggregation, Final recommendation |

### Key Patterns Demonstrated

1. **Multi-Agent Orchestration** - LangGraph StateGraph for agent coordination
2. **Shared State** - Agents accumulate findings in shared state
3. **Tool Reuse** - Shared tools (sanctions, policies, customers)
4. **Sequential with Context** - Each agent builds on previous findings
5. **Safety Guardrails** - Validation against forbidden phrases

### Comparison with Previous Notebooks

| Feature | Notebook 2 (Sequential) | Notebook 3 (Router) | Notebook 4 (Multi-Agent) |
|---------|------------------------|--------------------|--------------------------|
| Flow | Linear A→B→C | Conditional routing | Agent collaboration |
| State | Minimal | Per-transaction | Shared & accumulated |
| Complexity | Simple | Medium | Advanced |
| Use Case | Single workflow | Type-based routing | Complex decisions |

### Key Principle

**AI agents collaborate and share information, but ONLY human analysts make final payment decisions.**

### Extensions

- Add parallel agent execution for performance
- Implement agent voting/consensus mechanisms
- Add human-in-the-loop nodes for critical decisions
- Integrate with real compliance databases
- Add agent memory for learning from past decisions